In [16]:
import re
import pyrfume
import numpy as np
import pandas as pd
import seaborn as sns
import rdkit.Chem as Chem
import matplotlib.pyplot as plt
from rdkit.Chem import rdMolDescriptors
from sklearn.preprocessing import MultiLabelBinarizer
from computer_ontology.custom_funcs import *
from computer_ontology.config import *

In [17]:
def computer_taxonomy_imposer(replace, dataset):
  umbrella_dataset = dataset[['CID', 'IsomericSMILES', 'Descriptors']].copy()
  umbrella_dataset['Descriptors'] = dataset['Descriptors'].apply(lambda x: ';'.join([item for item in x.split(';') if item not in labels_to_remove.index]))
  umbrella_dataset = umbrella_dataset[umbrella_dataset['Descriptors'] != '']
  # Replacing the odor descriptors with the umbrella terms
  umbrella_dataset['Descriptors'] = umbrella_dataset['Descriptors'].apply(check_and_replace, replace = replace)
  umbrella_dataset['Descriptors'] = umbrella_dataset['Descriptors'].apply(make_unique)
  umbrella_dataset['Descriptors'] = umbrella_dataset['Descriptors'].dropna()
  umbrella_dataset = umbrella_dataset[umbrella_dataset['Descriptors'] != '']
  #Turn descriptors column into lists
  umbrella_dataset['Descriptors'] = umbrella_dataset['Descriptors'].apply(lambda x: x.split(';'))

  mlb = MultiLabelBinarizer(sparse_output=True)
  mlb.fit(umbrella_dataset['Descriptors'])

  data_bin = umbrella_dataset.join(pd.DataFrame.sparse.from_spmatrix(mlb.transform(umbrella_dataset['Descriptors']), index=umbrella_dataset.index, columns=mlb.classes_))
  return data_bin

In [18]:
train = pd.read_csv(alldesc_train_path_2025)
test = pd.read_csv(alldesc_test_path_2025)

In [19]:
labels_df = pd.read_excel(computer_tax_path, index_col=0)

# Labels that did not cluster with any other labels were labelled as 'NaN'
labels_to_remove = labels_df[labels_df['Umbrella Terms'].isna()]['Original Descriptors']
labels_df = labels_df.dropna()
replace = labels_df.values.tolist()

In [21]:
train = computer_taxonomy_imposer(replace, train)
test = computer_taxonomy_imposer(replace, test)

In [ ]:
train.to_csv(computer_train_path_2025, index=False)
test.to_csv(computer_test_path_2025, index=False)